# WP-2 Car Adjustment

<p style="color:rgb(0,162,219); font-family:Arial; font-size:16px;">Notebook Information </p>

<table style="color:rgb(88,89,91); font-family:Arial; float:left; font-size:13px; text-align:left;">
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Project</b></td>
        <td style="text-align:left;">NorMITs Demand Partner </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Primary Contact Name</b></td>
        <td style="text-align:left;"> </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Primary Contact Email</b></td>
        <td style="text-align:left;"> </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Document Sensitivity</b></td>
        <td style="text-align:left;"> </td>
    </tr>
</table>

In [1]:
import pandas as pd 
import numpy as np

### Step 1 - Adjust inputs to create constraints

In [2]:
Tour_model_output = pd.read_csv(r'I:\NTS\outputs\tour\reports\v3\matrix_county_output.csv')
Tour_model_output.head()

,tmz_o,tmz_d,mode,purpose,direction,period,trips
0,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,1,192249.968341
1,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,2,82663.316371
2,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,3,30406.370034
3,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,4,51696.947005
4,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,5,25580.167447


In [3]:
bespoke_sectors = pd.read_csv(r'I:\NTS\imports\tour_adjust_imports\bespoke_sectors_v1.1.csv')
bespoke_sectors.head()

,STB,county,Region,Sector_ID,Sector,Level
0,TfN,Cheshire,North West,1,Cheshire,1
1,TfN,Cleveland,North East,2,Cleveland,1
2,TfN,Cumbria,North West,3,Cumbria,1
3,TfN,Durham,North East,4,Durham,1
4,TfN,Greater Manchester,North West,5,Greater Manchester,1


In [4]:
Tour_model_output = pd.merge(Tour_model_output, bespoke_sectors, left_on= 'tmz_o' , right_on= 'county' , how= 'inner')
Tour_model_output = Tour_model_output.rename(columns={'Sector_ID':'Sector_ID_o', 'Sector':'Sector_o'}).drop(columns=['STB','county','Level','Region'])
Tour_model_output = pd.merge(Tour_model_output, bespoke_sectors, left_on= 'tmz_d' , right_on= 'county' , how= 'inner')
Tour_model_output = Tour_model_output.rename(columns={'Sector_ID':'Sector_ID_d', 'Sector':'Sector_d'}).drop(columns=['STB','county','Level','Region'])

In [5]:
Tour_model_output = Tour_model_output.groupby(['mode', 'purpose', 'direction', 'period', 'Sector_o', 'Sector_ID_o', 'Sector_d', 'Sector_ID_d']).sum('trips')

In [6]:
Tour_model_output=Tour_model_output.reset_index()
Tour_model_output

,mode,purpose,direction,period,Sector_o,Sector_ID_o,Sector_d,Sector_ID_d,trips
0,Air,Commuting,hb_fr,1,EEH,18,Cheshire,1,149.679401
1,Air,Commuting,hb_fr,1,Greater Manchester,5,TfSE,20,30.027875
2,Air,Commuting,hb_fr,1,London,23,Scotland,24,125.107890
3,Air,Commuting,hb_fr,1,Scotland,24,TfSE,20,45.753334
4,Air,Commuting,hb_fr,3,East Midlands South,15,Scotland,24,22.535722
...,...,...,...,...,...,...,...,...,...
60938,Walk,Visit friends,nhb,6,Tyne and Wear,12,Tyne and Wear,12,9181.507845
60939,Walk,Visit friends,nhb,6,West Midlands North,16,West Midlands North,16,22866.228867
60940,Walk,Visit friends,nhb,6,West Midlands South,17,West Midlands South,17,20068.064635
60941,Walk,Visit friends,nhb,6,West Yorkshire,13,West Yorkshire,13,17677.401592


### Step 2 - Create Sector Constraints

In [7]:
df = Tour_model_output.copy()

In [8]:
# Function to create proportion column to calculate the proportion of total trips based on the direction
def calc_props(df, dirr):
    if (dirr == 'hb_fr') | (dirr == 'nhb'):
        df_total = df.groupby(['direction','mode','purpose','period','Sector_o','Sector_ID_o'])['trips'].sum().reset_index()
        df_total.rename(columns={'trips':'total_trips'}, inplace=True)
        df = pd.merge(df, df_total, on=['direction','mode','purpose','period','Sector_o','Sector_ID_o'], how='left')
        df['proportion'] = df.trips / df.total_trips
    else: 
        df_total = df.groupby(['direction','mode','purpose','period','Sector_d','Sector_ID_d'])['trips'].sum().reset_index()
        df_total.rename(columns={'trips':'total_trips'}, inplace=True)
        df = pd.merge(df, df_total, on=['direction','mode','purpose','period','Sector_d','Sector_ID_d'], how='left')
        df['proportion'] = df.trips / df.total_trips
    return df

#Call function for filtered directions
result_list = []
for dirr in ['hb_fr','nhb', 'hb_to']:
    df_filter = df.loc[df['direction'] == dirr]
    df_out = calc_props(df_filter, dirr)
    result_list.append(df_out)
    
# Append all filtered directions
result_df = pd.concat(result_list, ignore_index=True)


In [9]:
df = result_df

# Create marker column to group "Rest"
def marker(row):
    if row['direction'] in ['hb_fr', 'nhb']:
        if row['proportion'] < 0.05:
            return 'Rest'
        else:
            return row['Sector_d']
    elif row['direction'] == 'hb_to':
        if row['proportion'] < 0.05:
            return 'Rest'
        else:
            return row['Sector_o']

df['Marker'] = df.apply(marker, axis=1)

# sector_marker to be used to later to calculate trips constraints

sector_marker = df.loc[df.direction == 'hb_fr']
 
sector_marker = sector_marker[['Sector_o','Sector_d','Marker']].drop_duplicates()

df

,mode,purpose,direction,period,Sector_o,Sector_ID_o,Sector_d,Sector_ID_d,trips,total_trips,proportion,Marker
0,Air,Commuting,hb_fr,1,EEH,18,Cheshire,1,149.679401,149.679401,1.000000,Cheshire
1,Air,Commuting,hb_fr,1,Greater Manchester,5,TfSE,20,30.027875,30.027875,1.000000,TfSE
2,Air,Commuting,hb_fr,1,London,23,Scotland,24,125.107890,125.107890,1.000000,Scotland
3,Air,Commuting,hb_fr,1,Scotland,24,TfSE,20,45.753334,45.753334,1.000000,TfSE
4,Air,Commuting,hb_fr,3,East Midlands South,15,Scotland,24,22.535722,22.535722,1.000000,Scotland
...,...,...,...,...,...,...,...,...,...,...,...,...
60938,Walk,Visit friends,hb_to,6,Tyne and Wear,12,Tyne and Wear,12,33333.592647,33333.592647,1.000000,Tyne and Wear
60939,Walk,Visit friends,hb_to,6,West Midlands North,16,West Midlands North,16,56502.673507,56502.673507,1.000000,West Midlands North
60940,Walk,Visit friends,hb_to,6,West Midlands South,17,West Midlands South,17,92176.099549,92176.099549,1.000000,West Midlands South
60941,Walk,Visit friends,hb_to,6,West Yorkshire,13,West Yorkshire,13,96775.448979,96775.448979,1.000000,West Yorkshire


In [10]:
# Filter data for direction 'hb_fr' or 'nhb' (df 1)
df1 = df[(df['direction'] == 'hb_fr') | (df['direction'] == 'nhb')]

# Filter data for direction 'hb_to' (df 2)
df2 = df[df['direction'] == 'hb_to']

# Group df 1 by 'origin' and 'marker', summing 'trips' and 'proportion'
grouped_df1 = df1.groupby(['Sector_o', 'Marker','direction','purpose', 'mode', 'period'], as_index=False).agg({
    'trips': 'sum',
    'proportion': 'sum'
      
})

# Group df 2 by 'destination' and 'marker', summing 'trips' and 'proportion'
grouped_df2 = df2.groupby(['Sector_d', 'Marker', 'direction','purpose', 'mode', 'period'], as_index=False).agg({
   'trips': 'sum',
    'proportion': 'sum'   
})

grouped_df1['Sector_d'] = grouped_df1['Marker']
grouped_df2['Sector_o'] = grouped_df2['Marker']

grouped_df1 = grouped_df1.drop(columns=['Marker'])
grouped_df2 = grouped_df2.drop(columns=['Marker'])
                                                                        

# Append grouped datasets 1 and 2
appended_dataset = pd.concat([grouped_df1, grouped_df2], ignore_index=True)

In [11]:
sector_constraint = appended_dataset
sector_constraint

,Sector_o,direction,purpose,mode,period,trips,proportion,Sector_d
0,Cheshire,hb_fr,Commuting,Bus,1,26056.515303,0.913339,Cheshire
1,Cheshire,hb_fr,Commuting,Bus,2,5582.668085,0.829203,Cheshire
2,Cheshire,hb_fr,Commuting,Bus,3,423.683085,1.000000,Cheshire
3,Cheshire,hb_fr,Commuting,Bus,4,3185.848693,0.794743,Cheshire
4,Cheshire,hb_fr,Commuting,Bus,5,2098.596900,0.849857,Cheshire
...,...,...,...,...,...,...,...,...
38146,Western Gateway,hb_to,Visit friends,Walk,2,104192.642478,1.000000,Western Gateway
38147,Western Gateway,hb_to,Visit friends,Walk,3,87322.097960,1.000000,Western Gateway
38148,Western Gateway,hb_to,Visit friends,Walk,4,86272.843587,0.997678,Western Gateway
38149,Western Gateway,hb_to,Visit friends,Walk,5,80874.631656,1.000000,Western Gateway


#### Checks 

#### Check 1 - Check min and max of the proportion column. Make sure between 0 and 1.

In [12]:
min_proportion = sector_constraint['proportion'].min()
min_proportion = '{:,.6f}'.format(min_proportion)
min_proportion

'0.000121'

In [13]:
max_proportion = sector_constraint['proportion'].max()
max_proportion

1.0

#### Check 2 - Check total trips in appended_dataset = total trips in Tour_model_output

In [14]:
total_tour_model_trips = Tour_model_output['trips'].sum()
total_sector_constraint_trips = sector_constraint['trips'].sum()

if total_tour_model_trips == total_sector_constraint_trips:
    print('Total trips match')
else:
    print('Total trips do not match')

Total trips do not match


In [15]:
total_tour_model_trips

1029263266.1297952

In [16]:
total_sector_constraint_trips

1029263266.1297951

#### Check 3 - Check the proportion column sums to 1 if you groupby:
['Sector_o', 'direction', 'purpose', 'mode', 'period'] for direction = hb_fr, nhb

['Sector_d', 'direction', 'purpose', 'mode', 'period'] for direction = hb_to

In [17]:
grouped_df1 = df1.groupby(['Sector_o','direction','purpose', 'mode', 'period'])['proportion'].sum().reset_index()
grouped_df1['proportion'] = grouped_df1['proportion'].astype(float)
grouped_df1

,Sector_o,direction,purpose,mode,period,proportion
0,Cheshire,hb_fr,Commuting,Bus,1,1.0
1,Cheshire,hb_fr,Commuting,Bus,2,1.0
2,Cheshire,hb_fr,Commuting,Bus,3,1.0
3,Cheshire,hb_fr,Commuting,Bus,4,1.0
4,Cheshire,hb_fr,Commuting,Bus,5,1.0
...,...,...,...,...,...,...
13184,Western Gateway,nhb,Visit friends,Walk,2,1.0
13185,Western Gateway,nhb,Visit friends,Walk,3,1.0
13186,Western Gateway,nhb,Visit friends,Walk,4,1.0
13187,Western Gateway,nhb,Visit friends,Walk,5,1.0


In [18]:
grouped_df2 = df2.groupby(['Sector_d','direction','purpose', 'mode', 'period'])['proportion'].sum().reset_index()
grouped_df2['proportion'] = grouped_df2['proportion'].astype(float)
grouped_df2

,Sector_d,direction,purpose,mode,period,proportion
0,Cheshire,hb_to,Commuting,Bus,1,1.0
1,Cheshire,hb_to,Commuting,Bus,2,1.0
2,Cheshire,hb_to,Commuting,Bus,3,1.0
3,Cheshire,hb_to,Commuting,Bus,4,1.0
4,Cheshire,hb_to,Commuting,Bus,5,1.0
...,...,...,...,...,...,...
6896,Western Gateway,hb_to,Visit friends,Walk,2,1.0
6897,Western Gateway,hb_to,Visit friends,Walk,3,1.0
6898,Western Gateway,hb_to,Visit friends,Walk,4,1.0
6899,Western Gateway,hb_to,Visit friends,Walk,5,1.0


In [19]:
#Check for sectors where proportion != 1 considering a tolerance level of 0.000001 (i.e., proportion is between 0.999999 and 1.000001)
grouped_df1_filtered = grouped_df1[~grouped_df1['proportion'].between(1-1e-6, 1+1e-6)]
grouped_df1_filtered

,Sector_o,direction,purpose,mode,period,proportion


In [20]:
#Check for sectors where proportion != 1 considering a tolerance level of 0.000001 (i.e., proportion is between 0.999999 and 1.000001)
grouped_df2_filtered = grouped_df2[~grouped_df2['proportion'].between(1-1e-6, 1+1e-6)]
grouped_df2_filtered

,Sector_d,direction,purpose,mode,period,proportion


### Step 3: Apply Constraints

In [21]:
#Read in od/pa sectorised synthetic data

synth_data = pd.read_csv(r'I:\NTS\imports\tour_adjust_imports\sectorised_hb_synthetic_pa_yr2018_p1_m3.csv')
synth_data

,Sector_o,Sector_d,synth_trips
0,Cheshire,Cheshire,9.306605e+05
1,Cheshire,Cleveland,1.572779e+01
2,Cheshire,Cumbria,1.522151e+03
3,Cheshire,Durham,2.466047e+01
4,Cheshire,EEH,1.997885e+02
...,...,...,...
671,Western Gateway,Tyne and Wear,2.647929e+00
672,Western Gateway,West Midlands North,1.251536e+03
673,Western Gateway,West Midlands South,3.950396e+04
674,Western Gateway,West Yorkshire,3.393207e+01


In [22]:
# Read in look up for mode and purpose IDs for the purpose of referencing

mode_lookup = pd.read_csv(r'I:\NTS\classified builds\lookups\luk_mode.csv').drop(columns=['mainmode_b11id', 'mainmode_b11id_desc']).drop_duplicates().rename(columns={'mode':'mode_id'})
purpose_lookup = pd.read_csv(r'I:\NTS\classified builds\lookups\luk_purpose.csv').drop(columns=['mainmode_b11id','trippurpfrom_b01id','trippurpto_b01id', 'mainmode_b11id_desc','trippurpfrom_b01id_desc','trippurpto_b01id_desc']).drop_duplicates().rename(columns={'purpose':'purpose_id'})

In [23]:
# Join purpose and mode look up with sector_constraint to filter by IDs rather than names

sector_constraint = pd.merge(sector_constraint, mode_lookup, left_on= 'mode', right_on= 'mode_name' , how='inner' )
sector_constraint = pd.merge(sector_constraint, purpose_lookup, left_on= 'purpose', right_on= 'purpose_name' , how='inner' )

In [24]:
#Filter sector_constraint for purpose = 1, mode = 3, direction = hb_fr, period = 1 

filtered_sector_constraint = sector_constraint[(sector_constraint['direction']== 'hb_fr')& 
                                              (sector_constraint['purpose_id']==1)&
                                              (sector_constraint['mode_id']==3)&
                                              (sector_constraint['period']== 1)]
filtered_sector_constraint=filtered_sector_constraint.drop(columns=['mode_id', 'purpose_id', 'direction','purpose','mode','period','mode_name','purpose_name'])

filtered_sector_constraint

,Sector_o,trips,proportion,Sector_d
762,Cheshire,5.277026e+05,0.794055,Cheshire
774,Cheshire,6.922233e+04,0.104162,Greater Manchester
791,Cheshire,6.764179e+04,0.101783,Rest
804,Cleveland,2.578464e+05,0.872953,Cleveland
816,Cleveland,1.771244e+04,0.059966,Durham
...,...,...,...,...
1490,West Midlands South,1.715906e+06,0.919819,West Midlands South
1503,West Yorkshire,8.405102e+04,0.069183,Rest
1514,West Yorkshire,1.130863e+06,0.930817,West Yorkshire
1528,Western Gateway,1.909926e+05,0.097145,Rest


In [25]:
# constraint_creator from above merged with the sectorised synthetic data to create the Marker column. 
sector_cons = pd.merge(sector_marker, synth_data, on= ['Sector_o', 'Sector_d'], how= 'left')
sector_cons

,Sector_o,Sector_d,Marker,synth_trips
0,EEH,Cheshire,Cheshire,305.062211
1,Greater Manchester,TfSE,TfSE,33.264882
2,London,Scotland,Scotland,53.029746
3,Scotland,TfSE,TfSE,3.656745
4,East Midlands South,Scotland,Scotland,21.216866
...,...,...,...,...
1103,London,Western Gateway,Western Gateway,5530.122138
1104,EEH,Tyne and Wear,Tyne and Wear,26.949577
1105,North Wales,Western Gateway,Western Gateway,2437.615473
1106,London,West Midlands South,West Midlands South,2950.634230


In [26]:
# Created a proportions column by keeping the proportion column from sector_constraint. This will be used to calculate the proportion of the synth_trips

proportion_sector_constraint = pd.merge(sector_cons, filtered_sector_constraint, left_on= ['Sector_o', 'Marker'],right_on= ['Sector_o', 'Sector_d'], how='left')
proportion_sector_constraint = proportion_sector_constraint.fillna(0).drop(columns=['Sector_d_y']).rename(columns={'Sector_d_x':'Sector_d'})
proportion_sector_constraint[proportion_sector_constraint['Marker']=='Rest']

,Sector_o,Sector_d,Marker,synth_trips,trips,proportion
27,Scotland,EEH,Rest,8.907735e+00,4124.997674,0.001456
29,Scotland,Scotland,Rest,6.964058e+06,4124.997674,0.001456
47,London,Northumberland,Rest,1.918282e+00,176633.578895,0.076711
51,Scotland,East Midlands South,Rest,7.408197e+00,4124.997674,0.001456
53,Scotland,North Wales,Rest,7.566674e+00,4124.997674,0.001456
...,...,...,...,...,...,...
1047,TfSE,Durham,Rest,7.594464e+00,100799.737004,0.023767
1080,TfSE,Northumberland,Rest,2.975956e+00,100799.737004,0.023767
1085,North Wales,Cumbria,Rest,1.577788e+03,21641.036363,0.043418
1088,Humberside,Western Gateway,Rest,2.205918e+01,37903.325704,0.079619


In [27]:
synth_trips_total_od = proportion_sector_constraint.groupby(['Sector_o','Marker'])['synth_trips'].sum().reset_index()
synth_trips_total_od = synth_trips_total_od.rename(columns={'synth_trips': 'synth_trips_total_od'})
synth_trips_total_od = pd.merge(proportion_sector_constraint, synth_trips_total_od, on=['Sector_o', 'Marker'], how='left')
synth_trips_total_od

,Sector_o,Sector_d,Marker,synth_trips,trips,proportion,synth_trips_total_od
0,EEH,Cheshire,Cheshire,305.062211,0.0,0.0,305.062211
1,Greater Manchester,TfSE,TfSE,33.264882,0.0,0.0,33.264882
2,London,Scotland,Scotland,53.029746,0.0,0.0,53.029746
3,Scotland,TfSE,TfSE,3.656745,0.0,0.0,3.656745
4,East Midlands South,Scotland,Scotland,21.216866,0.0,0.0,21.216866
...,...,...,...,...,...,...,...
1103,London,Western Gateway,Western Gateway,5530.122138,0.0,0.0,5530.122138
1104,EEH,Tyne and Wear,Tyne and Wear,26.949577,0.0,0.0,26.949577
1105,North Wales,Western Gateway,Western Gateway,2437.615473,0.0,0.0,2437.615473
1106,London,West Midlands South,West Midlands South,2950.634230,0.0,0.0,2950.634230


In [28]:
synth_trips_total_od['synth_prop'] = (synth_trips_total_od['synth_trips']/synth_trips_total_od['synth_trips_total_od'])*synth_trips_total_od['proportion']
synth_trips_total_od

,Sector_o,Sector_d,Marker,synth_trips,trips,proportion,synth_trips_total_od,synth_prop
0,EEH,Cheshire,Cheshire,305.062211,0.0,0.0,305.062211,0.0
1,Greater Manchester,TfSE,TfSE,33.264882,0.0,0.0,33.264882,0.0
2,London,Scotland,Scotland,53.029746,0.0,0.0,53.029746,0.0
3,Scotland,TfSE,TfSE,3.656745,0.0,0.0,3.656745,0.0
4,East Midlands South,Scotland,Scotland,21.216866,0.0,0.0,21.216866,0.0
...,...,...,...,...,...,...,...,...
1103,London,Western Gateway,Western Gateway,5530.122138,0.0,0.0,5530.122138,0.0
1104,EEH,Tyne and Wear,Tyne and Wear,26.949577,0.0,0.0,26.949577,0.0
1105,North Wales,Western Gateway,Western Gateway,2437.615473,0.0,0.0,2437.615473,0.0
1106,London,West Midlands South,West Midlands South,2950.634230,0.0,0.0,2950.634230,0.0


In [29]:
trips_constraint = synth_trips_total_od.groupby(['Sector_o'])['synth_trips'].sum().reset_index()
trips_constraint = trips_constraint.rename(columns={'synth_trips': 'synth_trips_o'})
trips_constraint = pd.merge(synth_trips_total_od, trips_constraint, on=['Sector_o'], how='left')
trips_constraint

,Sector_o,Sector_d,Marker,synth_trips,trips,proportion,synth_trips_total_od,synth_prop,synth_trips_o
0,EEH,Cheshire,Cheshire,305.062211,0.0,0.0,305.062211,0.0,1.329256e+07
1,Greater Manchester,TfSE,TfSE,33.264882,0.0,0.0,33.264882,0.0,3.607037e+06
2,London,Scotland,Scotland,53.029746,0.0,0.0,53.029746,0.0,8.383140e+06
3,Scotland,TfSE,TfSE,3.656745,0.0,0.0,3.656745,0.0,1.398171e+07
4,East Midlands South,Scotland,Scotland,21.216866,0.0,0.0,21.216866,0.0,2.590490e+06
...,...,...,...,...,...,...,...,...,...
1103,London,Western Gateway,Western Gateway,5530.122138,0.0,0.0,5530.122138,0.0,8.383140e+06
1104,EEH,Tyne and Wear,Tyne and Wear,26.949577,0.0,0.0,26.949577,0.0,1.329256e+07
1105,North Wales,Western Gateway,Western Gateway,2437.615473,0.0,0.0,2437.615473,0.0,1.362771e+06
1106,London,West Midlands South,West Midlands South,2950.634230,0.0,0.0,2950.634230,0.0,8.383140e+06


In [30]:
trips_constraint['trips_constraint'] = trips_constraint['synth_trips_o']*trips_constraint['synth_prop']
trips_constraint

,Sector_o,Sector_d,Marker,synth_trips,trips,proportion,synth_trips_total_od,synth_prop,synth_trips_o,trips_constraint
0,EEH,Cheshire,Cheshire,305.062211,0.0,0.0,305.062211,0.0,1.329256e+07,0.0
1,Greater Manchester,TfSE,TfSE,33.264882,0.0,0.0,33.264882,0.0,3.607037e+06,0.0
2,London,Scotland,Scotland,53.029746,0.0,0.0,53.029746,0.0,8.383140e+06,0.0
3,Scotland,TfSE,TfSE,3.656745,0.0,0.0,3.656745,0.0,1.398171e+07,0.0
4,East Midlands South,Scotland,Scotland,21.216866,0.0,0.0,21.216866,0.0,2.590490e+06,0.0
...,...,...,...,...,...,...,...,...,...,...
1103,London,Western Gateway,Western Gateway,5530.122138,0.0,0.0,5530.122138,0.0,8.383140e+06,0.0
1104,EEH,Tyne and Wear,Tyne and Wear,26.949577,0.0,0.0,26.949577,0.0,1.329256e+07,0.0
1105,North Wales,Western Gateway,Western Gateway,2437.615473,0.0,0.0,2437.615473,0.0,1.362771e+06,0.0
1106,London,West Midlands South,West Midlands South,2950.634230,0.0,0.0,2950.634230,0.0,8.383140e+06,0.0


#### Checks 

#### Check 1 - Synth_trips sum should equal trips_constraint sum

In [31]:
synth_trip = trips_constraint['synth_trips'].sum()
trips_cons = trips_constraint['trips_constraint'].sum()

if synth_trip == trips_cons:
    print('Total trips match')
else:
    print('Total trips do not match')

Total trips match


In [32]:
trips_constraint['synth_trips'].sum()

112186504.15061027

In [33]:
trips_constraint['trips_constraint'].sum()

112186504.15061027

#### Check 2 - Check that synth_prop sums to 1 when grouped by Sector_o

In [34]:
grouped_synth_prop = trips_constraint.groupby(['Sector_o'])['synth_prop'].sum().reset_index()
grouped_synth_prop['synth_prop'] = grouped_synth_prop['synth_prop'].astype(float)
grouped_synth_prop

,Sector_o,synth_prop
0,Cheshire,1.0
1,Cleveland,1.0
2,Cumbria,1.0
3,Durham,1.0
4,EEH,1.0
5,East Midlands North,1.0
6,East Midlands South,1.0
7,Greater Manchester,1.0
8,Humberside,1.0
9,Lancashire,1.0


In [36]:
grouped_synth_prop_check = grouped_synth_prop[~grouped_synth_prop['synth_prop'].between(1-1e-6, 1+1e-6)]
grouped_synth_prop_check

,Sector_o,synth_prop
